In [27]:
from abc import ABC, abstractmethod
from ecoli.shared.utils.schemas import get_config_schema, get_defaults_schema

class iX:
    defaults = {}
    config_schema = {}
    _ports = {
        "inputs": [],
        "outputs": []
    }

    def __init_subclass__(cls, **kwargs): 
        cls.config_schema = {
            **get_config_schema(cls.defaults),
            "time_step": {"_default": 1.0, "_type": "float"}
        }

    @property 
    def input_ports(self):
        return self._ports["inputs"]
    
    @input_ports.setter
    def input_ports(self, v):
        self._ports["inputs"] = v

    def ports_schema(self):
        pass

    def inputs(self):
        # extract from ports schema
        ports = self.ports_schema()
        if len(self.input_ports):
            ports = {
                port: ports[port]
                for port in self.input_ports
            }
        return get_defaults_schema(ports)


class X(iX):
    defaults = {'k': 11.11}

    def __init__(self):
        self.input_ports = ['x', 'y']

    def ports_schema(self):
        return {
            'x': {'_default': 11.11},
            'y': {'a': {'_default': 2.22}},
            'z': {'_default': 3}
        }
    

x = X()
x.inputs()

{'x': 'float', 'y': {'a': 'float'}}

In [33]:
class A:
    def schema(self):
        return {'y': 11.11}
        
    def __init__(self):
        self._port_data = self.schema()

    def inputs(self):
        import copy
        ports = copy.deepcopy(self._port_data)
        if not "x" in ports:
            ports["x"] = "float"
        return ports 


a = A()
a._port_data, a.inputs()
    

({'y': 11.11}, {'y': 11.11, 'x': 'float'})

In [7]:
from process_bigraph import pp

def extract_process_states(composer):
    composition = composer.generate()
    topology = composition['topology']
    processes = composition['processes']
    
    state = {}
    for processid, ports in topology.items():
        proc = processes[processid]
        process_state = {
            "_type": "step",
            "address": f"local:{processid}",
            "config": proc.config,
            "inputs": {},
            "outputs": {}
        }
        state[processid] = process_state

        inputs = proc.inputs()
        outputs = proc.outputs()
        for portname, store in ports.items():
            if portname in inputs:
                process_state["inputs"][portname] = store
            if portname in outputs:
                process_state["outputs"][portname] = store
                
    return state


def extract_store(proc, process_state: dict, topology_ports: dict, port_type: str):
    schema = getattr(proc, port_type)()
    for portname, store in topology_ports.items():
        if portname in schema:
            process_state[port_type][portname] = store


def test_state_extraction():
    from ecoli.shared.registry import ecoli_core as ec
    from ecoli.migrated.chromosome_structure import TestComposer
    from process_bigraph import Composite

    composer = TestComposer()
    composer.generate()['topology']['unique_update']
    state = extract_process_states(composer)
    pp(state)


test_state_extraction()

{ 'chromosome_structure': { '_type': 'step',
                            'address': 'local:chromosome_structure',
                            'config': { 'active_tfs': ['CPLX-125[c]'],
                                        'amino_acids': ['L-ALPHA-ALANINE[c]'],
                                        'calculate_superhelical_densities': True,
                                        'emit_unique': False,
                                        'fragmentBases': ['polymerized_ATP[c]'],
                                        'inactive_RNAPs': 'APORNAP-CPLX[c]',
                                        'mature_rna_end_positions': [],
                                        'mature_rna_ids': ['alaT-tRNA[c]'],
                                        'mature_rna_nt_counts': [],
                                        'n_TFs': 1,
                                        'n_TUs': 1,
                                        'n_amino_acids': 1,
                                        'n_fragment_ba

In [3]:
from ecoli.shared.utils.schemas import numpy_schema, UNIQUE_DIVIDERS


schema = numpy_schema("oriCs")
UNIQUE_DIVIDERS

{'active_ribosome': {'divider': 'ribosome_by_RNA',
  'topology': {'RNA': ('..', 'RNA'),
   'full_chromosome': ('..', 'full_chromosome'),
   'chromosome_domain': ('..', 'chromosome_domain'),
   'active_RNAP': ('..', 'active_RNAP')}},
 'full_chromosomes': {'divider': 'by_domain',
  'topology': {'full_chromosome': (),
   'chromosome_domain': ('..', 'chromosome_domain')}},
 'chromosome_domains': {'divider': 'by_domain',
  'topology': {'full_chromosome': ('..', 'full_chromosome'),
   'chromosome_domain': ()}},
 'active_replisomes': {'divider': 'by_domain',
  'topology': {'full_chromosome': ('..', 'full_chromosome'),
   'chromosome_domain': ('..', 'chromosome_domain')}},
 'oriCs': {'divider': 'by_domain',
  'topology': {'full_chromosome': ('..', 'full_chromosome'),
   'chromosome_domain': ('..', 'chromosome_domain')}},
 'promoters': {'divider': 'by_domain',
  'topology': {'full_chromosome': ('..', 'full_chromosome'),
   'chromosome_domain': ('..', 'chromosome_domain')}},
 'chromosomal_segmen

In [2]:
for process_id, process in composer.generate()['processes'].items():
    print(process_id, process.outputs())
    print()

chromosome_structure {'bulk': 'array[any|(0,)]', 'active_replisomes': 'array[any|(0,)]', 'chromosome_domains': 'array[any|(0,)]', 'active_RNAPs': 'array[any|(0,)]', 'RNAs': 'array[any|(0,)]', 'active_ribosome': 'array[any|(0,)]', 'promoters': 'array[any|(0,)]', 'DnaA_boxes': 'array[any|(0,)]', 'genes': 'array[any|(0,)]', 'chromosomal_segments': 'array[any|(0,)]', 'full_chromosomes': 'array[any|(0,)]', 'oriCs': 'array[any|(0,)]', 'next_update_time': {'_divide': 'set', '_type': 'float'}, 'listeners': {'rnap_data': {'n_total_collisions': 'integer', 'n_headon_collisions': 'integer', 'n_codirectional_collisions': 'integer', 'headon_collision_coordinates': 'list', 'codirectional_collision_coordinates': 'list', 'n_removed_ribosomes': 'integer', 'incomplete_transcription_events': 'array[integer|(1,)]', 'n_empty_fork_collisions': 'integer', 'empty_fork_collision_coordinates': 'list'}}}

unique_update {'active_replisomes': 'array[any|(0,)]', 'oriCs': 'array[any|(0,)]', 'chromosome_domains': 'arr